In [0]:
%run ./GlobalVariables

In [0]:
%run ./UtilityHandler

In [0]:
import subprocess # Install pymssql silently

subprocess.run(
    ["pip", "install", "pymssql"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

# Import pymssql and its error class
import pymssql
from pymssql import OperationalError

In [0]:
class ProcessConfig:
    def __init__(self, environment):
        self.environment = environment
        self.atrribute_table = 'cfg.AttributeSystem'
        self.global_vars = GlobalVars(environment)

    def get_connection_variable(self, p_key):
        return self.global_vars.get_database_config(p_key)

    def get_dbx_scope_value(self, p_key):
        return dbutils.secrets.get(scope=self.get_connection_variable('dbx_scope'), key=self.get_connection_variable(p_key))    

    def build_connection_url(self):
        v_connection_dict = {
            "hostname": self.get_connection_variable("hostname"),
            "port": self.get_connection_variable("port"),
            "database": self.get_connection_variable("database"),
            "options": self.get_connection_variable("options")
        }
        url_text =  f"""jdbc:sqlserver://{v_connection_dict['hostname']}:{v_connection_dict['port']};
                    database={v_connection_dict['database']};
                    {v_connection_dict['options']}
                """
        return StringUtils.remove_returns_and_blank_lines(url_text)

    def build_connection_properties_dict(self):
        
        v_connection_properties_dict = {
            "user"    : self.get_connection_variable('user'),
            "password": self.get_dbx_scope_value('kv_secret_name'),
            "driver": self.get_connection_variable('driver')       
        }

        return v_connection_properties_dict
    
    def get_config_table_data(self, p_table):
        v_url = self.build_connection_url()
        v_connection_properties_dict = self.build_connection_properties_dict()
        return spark.read.jdbc(url=v_url, table=p_table, properties=v_connection_properties_dict)
    
    def get_config_query_data(self, p_query):
        v_url = self.build_connection_url()
        v_connection_properties_dict = self.build_connection_properties_dict()
        v_query = f"""({p_query}) tmp"""
        return spark.read.jdbc(url=v_url, table=v_query, properties=v_connection_properties_dict)
    
    def get_config_attribute_value(self, p_attribute):
        df = self.get_config_query_data(f"""SELECT AttributeName, AttributeValue FROM {self.atrribute_table} WHERE AttributeName = '{p_attribute}' and Environment = '{self.environment}'""")
        return df.collect()[0]['AttributeValue'] if df.count() > 0 else None

    def get_config_attributes_value(self, p_attribute_array):
        quoted_attributes_array = [f"'{word}'" for word in p_attribute_array]
        p_attribute_in_str = ", ".join(quoted_attributes_array)

        df = self.get_config_query_data(f"""SELECT AttributeName, AttributeValue FROM {self.atrribute_table} WHERE AttributeName in ({p_attribute_in_str}) and Environment = '{self.environment}'""")

        return { row['AttributeName']: row['AttributeValue'] for row in df.collect()} if df.count() > 0 else None 

    def execute_config_stored_procedure(self, p_stored_procedure_name, p_parameters_dict=None):

        v_return = -1
        v_query = ""

        v_hostname = self.get_connection_variable('hostname')
        v_database = self.get_connection_variable('database')
        v_user = self.get_connection_variable('user')
        v_dbx_scope = self.get_connection_variable('dbx_scope')
        v_password = self.get_dbx_scope_value('kv_secret_name') 

        v_parameters_str = ""
        if p_parameters_dict is not None:

            if not isinstance(p_parameters_dict, dict):
                raise Exception(f"p_parameters_dict is not a dictionary.")

            for key, value in p_parameters_dict.items():
                v_parameters_str += f"@{key} = '{value}'"
                v_parameters_str += ", "
            v_parameters_str = v_parameters_str[:-2]

        v_query = f"EXEC {p_stored_procedure_name} {v_parameters_str}"

        try:
            with pymssql.connect(
                server=v_hostname,
                user=v_user,
                password=v_password,
                database=v_database
            ) as v_conn:
                with v_conn.cursor() as v_cursor:
                    v_cursor.execute(v_query)
                    v_conn.commit()
                    v_cursor.close()
                v_conn.close()
                v_return = 0
        except Exception as e:
            raise Exception(f"Error: {v_query}\n{e}")
        return v_return
   

In [0]:
class ProcessConfigData(ProcessConfig):
    def __init__(self, p_environment):
        super().__init__(p_environment)
        self.environment = p_environment
   
    def get_base_config_data(self, p_view_name, p_internal_product_id, p_server_name, p_database_name=None, p_internal_client_id = None, p_internal_facility_id = None, p_ingestion_pipeline_name = None):

        p_internal_client_id = None if p_internal_client_id == 0 else p_internal_client_id
        p_internal_facility_id = None if p_internal_facility_id == 0 else p_internal_facility_id

        v_query = f"""SELECT * FROM {p_view_name} WHERE SourceServerName1 = '{p_server_name}' and InternalProductId = {p_internal_product_id}"""
        if p_database_name is not None:
            v_query += f""" AND SourceDatabaseName1 = '{p_database_name}'"""
        if p_internal_client_id is not None:
            v_query += f""" AND InternalClientId = {p_internal_client_id}"""
        if p_internal_facility_id is not None:
            v_query += f""" AND InternalFacilityId = {p_internal_facility_id}"""
        if p_ingestion_pipeline_name is not None:
            v_query += f""" AND IngestionPipelineName = '{p_ingestion_pipeline_name}'"""
        return self.get_config_query_data(v_query)

    def get_server_list(self, p_internal_product_id, p_server_name, p_database_name=None, p_internal_client_id = None, p_internal_facility_id = None, p_ingestion_pipeline_name = None):
        v_view_name = 'cfg.vw_DataFabricServerList'
        return self.get_base_config_data(v_view_name, p_internal_product_id, p_server_name, p_database_name, p_internal_client_id, p_internal_facility_id, p_ingestion_pipeline_name)
    
    def get_table_list(self, p_internal_product_id, p_server_name, p_database_name=None, p_internal_client_id = None, p_internal_facility_id = None, p_ingestion_pipeline_name = None):
        v_view_name = 'cfg.vw_DataFabricTableList'
        return self.get_base_config_data(v_view_name, p_internal_product_id, p_server_name, p_database_name, p_internal_client_id, p_internal_facility_id, p_ingestion_pipeline_name)

    def get_table_list_aggregate(self, p_internal_product_id, p_server_name, p_database_name=None, p_internal_client_id = None, p_internal_facility_id = None, p_ingestion_pipeline_name = None):
        v_view_name = 'cfg.vw_DataFabricTableListAggregate'
        return self.get_base_config_data(v_view_name, p_internal_product_id, p_server_name, p_database_name, p_internal_client_id, p_internal_facility_id, p_ingestion_pipeline_name)
    
    def get_table_split_list(self, p_internal_product_id, p_server_name, p_database_name=None, p_internal_client_id = None, p_internal_facility_id = None, p_ingestion_pipeline_name = None):
        v_view_name = 'cfg.vw_DataFabricTableSplitList'
        return self.get_base_config_data(v_view_name, p_internal_product_id, p_server_name, p_database_name, p_internal_client_id, p_internal_facility_id, p_ingestion_pipeline_name)

    def set_pipeline_process_status_list(self, p_pipeline_list):
        for v_pipeline_dict in p_pipeline_list:
            self.set_pipeline_process_status(v_pipeline_dict)

    def set_data_load_process_status_detail_list(self, p_row_list):
        for v_row_dict in p_row_list:
            self.set_data_load_process_status_detail(v_row_dict)

    def set_data_load_process_status_detail(self, p_row_dict):
        self.execute_config_stored_procedure('dbo.usp_DataFabricUpdateProcessStatusFacilityDetail', p_row_dict)

    def set_pipeline_process_status(self, p_pipeline_dict):
        self.execute_config_stored_procedure('dbo.usp_DataFabricUpdateProcessStatusFacility', p_pipeline_dict)

    def set_pipeline_id(self, p_params_dict):
        self.execute_config_stored_procedure('cfg.usp_DataFabricUpdatePipelineConfig', p_params_dict)

        

In [0]:
#'AdlsAnalyticsFullpathUri','AdlsAnalyticsFullpathUriArchive','AdlsAnalyticsSecretScope','Environment','AnalyticsUnityCatalog'
#process_config = ProcessConfigImplementation('dev')
#print(process_config.get_config_attribute_value('AdlsUrl1'))
#print(process_config.get_config_attributes_value(['AdlsUrl', 'AdlsAnalyticsFullpathUri','AdlsAnalyticsFullpathUriArchive','AdlsAnalyticsSecretScope','Environment','AnalyticsUnityCatalog'])) 
#print(process_config.get_config_attribute_value('AdlsAnalyticsFullpathUri'))